In [11]:
!pip -q install langchain openai tiktoken chromadb

In [12]:
!pip show langchain

Name: langchain
Version: 0.1.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/nargizkaniyeva/Documents/Chatbot-AI/ai-legal-assistant/lib/python3.12/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files
- ChromaDB
- Source info
- gpt-4-turbo API

## Setting up LangChain


In [27]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-5sn307TKI2oB6sdpHoDGT3BlbkFJNaf8n1H03dgbxNFCjbRV"

In [46]:
import numpy as np
from stop_words import get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader


## Load multiple and process documents

In [17]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('articles/', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()

In [18]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [19]:
len(texts)

1096

In [21]:
texts[3]

Document(page_content='3) абонентская линия - линия связи, являющаяся частью местной сети телекоммуникаций и соединяющая абонентское устройство со средствами телекоммуникаций этой сети;\n\n\xa0\xa0\xa0\xa0\xa0 4) абонентское устройство - средство связи индивидуального использования, формирующее сигналы электрической связи для передачи или приема заданной абонентом информации и подключаемое к сети оператора связи;\n\n\xa0\xa0\xa0\xa0\xa0 4-1) перенос абонентского номера – услуга по сохранению и использованию абонентского номера в сетях сотовой связи, предоставляемая абоненту при заключении им нового договора об оказании услуг сотовой связи с другим оператором сотовой связи;\n\n\xa0\xa0\xa0\xa0\xa0 4-2) централизованная база данных абонентских номеров – аппаратно-программный комплекс управления базой данных, содержащей информацию об абонентских номерах сотовой связи, включая сведения, определяемые правилами переноса абонентского номера в сетях сотовой связи;', metadata={'source': 'articl

## create the DB

In [28]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [29]:
# persiste the db to disk
vectordb.persist()
vectordb = None

/Users/nargizkaniyeva/Documents/Chatbot-AI/ai-legal-assistant/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [30]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## Make a retriever

In [31]:
retriever = vectordb.as_retriever()

In [32]:
docs = retriever.get_relevant_documents("Кто имеет право запрашивать персональные данные РК?")

/Users/nargizkaniyeva/Documents/Chatbot-AI/ai-legal-assistant/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [33]:
len(docs)

4

In [34]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [35]:
retriever.search_type

'similarity'

In [36]:
retriever.search_kwargs

{'k': 2}

## Make a chain

In [37]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="map_reduce",
                                  retriever=retriever,
                                  return_source_documents=True)

/Users/nargizkaniyeva/Documents/Chatbot-AI/ai-legal-assistant/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [53]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\nИсточник:')
    
    # Create a set to keep track of unique sources to avoid duplicates
    unique_sources = set()
    
    # Initialize a counter for source numbering
    source_number = 1
    
    for source in llm_response["source_documents"]:
        source_path = source.metadata['source']
        source_title = os.path.basename(source_path).replace("_Cleaned.txt", "").replace("_", " ")
        
        # Check if the source has already been processed
        if source_path not in unique_sources:
            # Add the source to the set to track as seen
            unique_sources.add(source_path)
            
            # Print the source with numeration
            print(f"{source_number}. {source_title}")
            
            # Increment the source counter
            source_number += 1

In [54]:
# full example
query = "Кто имеет право запрашивать персональные данные РК?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Субъект или его законный представитель могут запрашивать персональные данные РК.

Источник:
1. О персональных данных и их защите
